## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#columns">Dropping Columns</a></li>
<li><a href="#sql">SQL Query</a></li>
<li><a href="#eda">Merging Dataframes </a></li>
<li><a href="#eda2">Merging Payment Dataframes </a></li>
<li><a href="#eda3">Merging Product Dataframes </a></li>
<li><a href="#data">Data Preprocessing</a></li>
</ul>

<a id='intro'></a>
## Introduction
This Notebook aims to preprocess and analyse a transaction dataset contained in different sheets in a excel file linked by primary and secondary keys. These sheets includes
- Orders
- Orderedproducts
- payments
- ProductDetails
- Reviews
- Customers

These Notebook are connected in the following ways (Through primary keys)
- Orders-> OrderedProducts (order_id)
- OrderedProducts-> Payments (order_id)
- OrderedProduct-> ProductDetails (product_id)
- Reviews-> Orders (order_id)
- customers-> Orders (customer_id)




In [104]:
import pandas as pd
import datetime
import numpy as np

# Loading the Dataframes from the excel file
df_orders = pd.read_excel('../data/dataset for bizops.xlsx', sheet_name= 'Orders')
df_orderprod = pd.read_excel('../data/dataset for bizops.xlsx', sheet_name= 'OrderedProducts')
df_payment = pd.read_excel('../data/dataset for bizops.xlsx', sheet_name= 'Payments')
df_prod_detail = pd.read_excel('../data/dataset for bizops.xlsx', sheet_name= 'Product Details')
df_reviews = pd.read_excel('../data/dataset for bizops.xlsx', sheet_name= 'Reviews')
df_customers = pd.read_excel('../data/dataset for bizops.xlsx', sheet_name= 'Customers')

<a id='columns'></a>

## Dropping Columns
Some columns are not needed for this analysis and it is always advised to drop unimportant columns to reduce the memory being used during analysis. This makes the code cells run faster.
The following cells would be dropped from the dataframes
- __df_odd_detail__ -> ('product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g',
                    'product_length_cm', 'product_height_cm', 'product_width_cm')

- __df_reviews__ -> ('review_creation_date','review_answer_timestamp',review_id)
- __df_customers__ -> ('customer_unique_id')



In [115]:
# Dropping unwanted columns in each dataframes

df_prod_detail.drop(['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g',
                    'product_length_cm', 'product_height_cm', 'product_width_cm'],
                     axis=1, inplace= True)


df_customers.drop(['customer_unique_id'], axis=1, inplace = True)


df_reviews.drop(['review_creation_date','review_answer_timestamp', 'review_id'], 
                axis=1, inplace = True)




<a id='eda'></a>
## Merging DataFrames 
All dataframs are combined into a single dataframe that is used for the final analysis. The dataframes are merged using their primary keys 
There will be two merged dataframes;
- One for the product
- One for the orders

__Orders__ will be merged with reviews, payment and customer tables


## Sql equivalent statement
This merge can be done in sql using the following command
```SQL
select * from orders
LEFT JOIN reviews ON orders.order_id = reviews.order_id
LEFT JOIN payment ON orders.order_id = payment.order_id
LEFT JOIN customer ON orders.order_id = customer.order_id
```

<a id='sql'></a>
## Sql equivalent statement
This merge can be done in sql using the following command
```SQL
select * from orders
LEFT JOIN reviews ON orders.order_id = reviews.order_id
LEFT JOIN payment ON orders.order_id = payment.order_id
LEFT JOIN customer ON orders.order_id = customer.order_id
```

In [116]:
#Merging the data with the customers dataframe
df_merge= df_orders.merge(df_customers, how='left', on= 'customer_id' )


df_merge.shape

(54011, 11)

The reviews dataframe contains duplicated rows, these rows will be dropped by using the _drop_duplicates__ method in pandas library
- Removing duplicates ensures that new rows are not created when a join happens between two dataframes

In [106]:
df_reviews.columns

Index(['review_id', 'order_id', 'review_score', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [117]:
print(f'The total rows in df_review_table is {df_reviews.shape[0]}')

duplicates= df_reviews.duplicated(subset=['order_id', 'review_score']).sum()
print(f'The total number of duplicated columns is {duplicates}')

The total rows in df_review_table is 56584
The total number of duplicated columns is 144


In [118]:
df_reviews = df_reviews.drop_duplicates(subset= ["order_id"])
df_reviews.shape


(56355, 2)

In [119]:
df_merge = df_merge.merge(df_reviews, how= 'left',  on= 'order_id')

df_merge.shape

(54011, 12)

<a id='eda2'></a>
## Merging the Payment Dataframe
Some products were paid for over a certain period, so to get the the total payment we have to
- Groupby the order_id
- sum by the total payment


In [120]:
print(f'The total rows in df_review_table is {df_payment.shape[0]}')

duplicates= df_payment.duplicated(subset=['order_id']).sum()
print(f'The total number of duplicated columns is {duplicates}')

The total rows in df_review_table is 54011
The total number of duplicated columns is 0


In [121]:
df_payment= df_payment.groupby('order_id').agg({'payment_sequential': 'first',
                                    'payment_type': 'first',
                                    'payment_installments': 'first',
                                    'payment_value': "sum"
                                    }).reset_index()


df_payment.shape

(54011, 5)

In [122]:
df_merge = df_merge.merge(df_payment, how= 'left',  on= 'order_id')

df_merge.shape

(54011, 16)

In [ ]:
df_merge.columns

In [124]:
df_merge.to_csv('../data/Order.csv', index= False)

<a id='eda3'></a>

## Merging Product Dataframe
This dataframe will consist of
- Order Product
- Product Table
- Orders Table
- Customer Table



In [126]:
df_orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [128]:
cols = ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 
        'order_delivered_customer_date', 'order_estimated_delivery_date']

df_prod_merge = df_orderprod.merge(df_orders[cols], how= 'left',  on= 'order_id')
df_prod_merge.shape

(61416, 11)

In [130]:
# Merging with Product table
df_prod_merge = df_prod_merge.merge(df_prod_detail, how= 'left',  on= 'product_id')
df_prod_merge.shape

(61416, 12)

In [132]:
df_prod_merge.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id', 'price',
       'freight_value', 'customer_id', 'order_status',
       'order_purchase_timestamp', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'product_category_name'],
      dtype='object')

In [133]:
df_customers.columns

Index(['customer_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state'],
      dtype='object')

In [134]:
#Merging with customer table

cols = ['customer_id',  'customer_city','customer_state']
df_prod_merge = df_prod_merge.merge(df_customers[cols], how= 'left',  on= 'customer_id')

df_prod_merge.shape

(61416, 14)

In [135]:
df_prod_merge.to_csv('../data/Products.csv', index= False)

<a id='data'></a>
## Data Preporocessing
Here we will carry out some data preporocession like
- Changing datetime columns todatetime objects
- Changing state names to their full names


In [155]:
# Loading the dataframes
df_pro = pd.read_csv('../data/Products.csv')
df_od = pd.read_csv('../data/Orders.csv')


In [156]:
datetime_list = [ 'order_purchase_timestamp', 'order_delivered_customer_date',
       'order_estimated_delivery_date']

for x in datetime_list:
    df_pro[x] = pd.to_datetime(df_pro[x])

df_pro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61416 entries, 0 to 61415
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       61416 non-null  object        
 1   order_item_id                  61416 non-null  int64         
 2   product_id                     61416 non-null  object        
 3   seller_id                      61416 non-null  object        
 4   price                          61416 non-null  float64       
 5   freight_value                  61416 non-null  float64       
 6   customer_id                    61416 non-null  object        
 7   order_status                   61416 non-null  object        
 8   order_purchase_timestamp       61416 non-null  datetime64[ns]
 9   order_delivered_customer_date  60319 non-null  datetime64[ns]
 10  order_estimated_delivery_date  61416 non-null  datetime64[ns]
 11  product_categor

In [ ]:
cols = [ 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date']

for x in cols:
    df_od[x] = pd.to_datetime(df_od[x])

df_od.info()

In [158]:
#extracting date and month from datetime columns in the Orders dataframe
import calendar

df_pro['order_month'] =df_pro['order_purchase_timestamp'].dt.month
df_pro['order_month'] = df_pro['order_month'].apply(lambda x: calendar.month_name[x])




In [161]:
#extracting date and month from datetime columns in the Product dataframe
df_od['order_month'] =df_od['order_purchase_timestamp'].dt.month
df_od['order_month'] = df_od['order_month'].apply(lambda x: calendar.month_name[x])



The states are abbreviated which causes ambiguity. A dataframe that contained the mappings between States codes and actual State name was downloaded from <a href="https://github.com/datasets-br/state-codes/blob/master/data/br-state-codes.csv"> Dataframe Link</a>.

This dataframe is used to map the State codes to the actual state names.


In [162]:
#Loading the downloaded dataframe and mappping the codes to the actual State name
df_states = pd.read_csv('../data/br-state-codes.csv')

df_od['customer_state']= df_od['customer_state'].map(df_states.set_index('subdivision')['name'])

df_od['customer_state']

0                São Paulo
1                São Paulo
2                São Paulo
3                São Paulo
4                São Paulo
               ...        
53770         Minas Gerais
53771       Rio de Janeiro
53772       Rio de Janeiro
53773    Rio Grande do Sul
53774       Rio de Janeiro
Name: customer_state, Length: 53775, dtype: object

In [163]:
#Doing the same for the product Dataframe
df_pro['customer_state']= df_pro['customer_state'].map(df_states.set_index('subdivision')['name'])

In [164]:
#Saving Off the tables to Dataframes
df_pro.to_csv('../data/Products.csv', index=False)
df_od.to_csv('../data/Order.csv', index=False)